<a href="https://colab.research.google.com/github/Niraj-Khatri/Pyspark-AWS/blob/master/Electronics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Spark and Import Dataset

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-driver

In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AmazonReviews").getOrCreate()

# Import Amazon Electronics Data File From AWS


In [4]:
# Import struct fields
from pyspark.sql.types import StructField, StringType, IntegerType, StructType, TimestampType

In [24]:


# Create schema of field types
schema = [StructField("marketplace", StringType(), True), StructField("customer_id", StringType(), True), StructField("review_id", StringType(), True), 
          StructField("product_id", StringType(), True), StructField("product_parent", StringType(), True),
          StructField("product_title", StringType(), True), StructField("product_category", StringType(), True), StructField("star_rating", IntegerType(), True),
          StructField("helpful_votes", IntegerType(), True), StructField("total_votes", IntegerType(), True), StructField("vine", StringType(), True),
          StructField("verified_purchase", StringType(), True), StructField("review_headline", StringType(), True), StructField("review_date", StringType(), False)]

final = StructType(fields=schema)

In [29]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.tsv.gz"), sep='\t', header=True)

# Clean the data
df = df.withColumn('review_date', df['review_date'].cast(TimestampType()))

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.|2015-08-31 00:00:00|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...| 

# Create Tables

## Customers Table

In [35]:
# Customer ID with customer view count
customers = df.groupBy("customer_id").count().show()


+-----------+-----+
|customer_id|count|
+-----------+-----+
|   11608395|    1|
|   22848554|    1|
|   43490052|    2|
|   16281356|    1|
|    3223564|    1|
|   44662810|    1|
|   41681532|    1|
|   16336729|    1|
|   31425737|    1|
|   31693763|    2|
|   12536186|    1|
|    7328230|    1|
|   52857540|    1|
|   40549446|    1|
|     485535|    1|
|    1350260|    1|
|   41257827|    2|
|   12374662|    9|
|   45193257|   39|
|     144481|    1|
+-----------+-----+
only showing top 20 rows



## Products Table


In [33]:
# Product ID and product title
products = df.groupby("product_id", "product_title").count().select("product_id", "product_title").show()


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00JL2498C|Jensen JTA-410-BL...|
|B00OUKG3NE|Sennheiser Androi...|
|B0083H9R6A|Electrified ELPLP...|
|B0097BEE9Q|Apple iPod nano 1...|
|B0037Z4MCM|Pyle Marine Sport...|
|B00P0BQKBC|Water & Wood Lock...|
|B00TP1C1UC|AmazonBasics 6-Ou...|
|B005AIF0X4|iHip MVF1030WO Ma...|
|B009JJKZSK|Click 'N Dig! Key...|
|B00KXDS4VQ|enKo products HDM...|
|B009W857SG|TETC Cable (3 Fee...|
|B007LG453Y|Tmvel TVM-UK2 Uni...|
|B00KFSO0SA|Monoprice 11149 3...|
|B00IH5U050|Digital Army Camo...|
|B00JDYMPYS|Sylvania SKCR2713...|
|B00YDMW0DQ|Sound Intone HD30...|
|B00AR0VCXW|Pro-Ject Speed Bo...|
|B006BE6WK6|12 PIECE Ear Hook...|
|B00TM1CT98|Bose Acoustimass ...|
|B00P8D911I|AuraBeam Samsung ...|
+----------+--------------------+
only showing top 20 rows



## Reviews Table


In [34]:
# The review number, who reviewed it, what he/she reviewed, the parent of the reviewed product, and the date of the review
reviews = df.select("review_id", "customer_id", "product_id", "product_parent", "review_date").show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R2MTG1GCZLR2DK|   41409413|B00428R89M|     112201306|2015-08-31 00:00:00|
|R2HBOEM8LE9928|   49668221|B000068O48|     734576678|2015-08-31 00:00:00|
|R1P4RW1R9FDPEE|   12338275|B000GGKOG8|     614448099|2015-08-31 00:00:00|
|R1EBPM82ENI67M|   38487968|B000NU4OTA|      72265257|2015-08-31 00:00:00|
|R372S58V6D11AT|   23732619|B00JOQIO6S|     308169188|2015-08-31 00:00:00|
|R1A4514XOYI1PD|   21257820|B008NCD2LG|     976385982|2015-08-31 00:00:00|
|R20D9EHB7N20V6|    3084991|B00007FGUF|     670878953|2015-08-31 00:00:00|
|R1WUTD8MVSROJU|    8153674|B00M9V2RMM|     508452933|2015-08-31 00:00:00|
|R1QCYLT25812DM|   52246189|B00J3O9DYI|     766372886|2015-08-31 00:00:00|
| R904DQPBCEM7A|   41463864|B00NS1A0E4|     458130381|2015-08-31 00:00:00|
|R1DGA6UQIVLKZ7|    27819

## Vines Table

In [36]:
# The review number, rating, helpful votes, total votes, and vine 
vines = df.select("review_id", "star_rating", "helpful_votes", "total_votes", "vine").show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2MTG1GCZLR2DK|          5|            0|          0|   N|
|R2HBOEM8LE9928|          5|            0|          0|   N|
|R1P4RW1R9FDPEE|          5|            1|          1|   N|
|R1EBPM82ENI67M|          1|            0|          0|   N|
|R372S58V6D11AT|          5|            1|          1|   N|
|R1A4514XOYI1PD|          5|            1|          1|   N|
|R20D9EHB7N20V6|          5|            0|          0|   N|
|R1WUTD8MVSROJU|          5|            0|          0|   N|
|R1QCYLT25812DM|          4|            0|          0|   N|
| R904DQPBCEM7A|          4|            0|          0|   N|
|R1DGA6UQIVLKZ7|          5|            0|          0|   N|
| RLQT3V8SMNIBH|          5|            0|          0|   N|
|R3T9GZS2TMXZGM|          1|            0|          0|   N|
|R24HVAEYP5PLDN|          5|            